In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack
from nltk.corpus import stopwords 

# import nltk
# nltk.download('stopwords')
print("done")

done


In [2]:
train_data = pd.read_csv("train.csv")


In [3]:
train_data[["text","retweets_count","favorites_count","followers_count","statuses_count","friends_count","verified","TweetID"]]

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,verified,TweetID
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,0,832509
1,populaire,0,0,86,1016,284,0,1388011
2,faut dégager cinglé,3,1,1944,28234,1995,0,63896
3,enseignants mettre prescriptions président rép...,0,0,1,1072,0,0,979251
4,mafieuse oppressive macron,0,0,13957,25311,10841,0,1040049
...,...,...,...,...,...,...,...,...
353963,gonflette tour raciste frustré,0,0,34,1509,55,0,142573
353964,france caste crapuleuse encadrée gangsters irr...,0,0,89,11166,127,0,240866
353965,eric zemmour français berbère,3,0,1888,712,3086,0,1173763
353966,gauchistes dépression pq,0,0,139,486,320,0,929182


In [4]:
temp= train_data[["text","retweets_count","favorites_count","followers_count","statuses_count","friends_count","verified","TweetID"]]

In [5]:
max(train_data["text"],key=len)

'radio france devient france radio france devient france tv inter france devient france tv régions france devient france tv juniors france devient france tv culture orange devient france télécom devient france macron république française devient france france'

In [6]:
temp

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,verified,TweetID
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,0,832509
1,populaire,0,0,86,1016,284,0,1388011
2,faut dégager cinglé,3,1,1944,28234,1995,0,63896
3,enseignants mettre prescriptions président rép...,0,0,1,1072,0,0,979251
4,mafieuse oppressive macron,0,0,13957,25311,10841,0,1040049
...,...,...,...,...,...,...,...,...
353963,gonflette tour raciste frustré,0,0,34,1509,55,0,142573
353964,france caste crapuleuse encadrée gangsters irr...,0,0,89,11166,127,0,240866
353965,eric zemmour français berbère,3,0,1888,712,3086,0,1173763
353966,gauchistes dépression pq,0,0,139,486,320,0,929182


In [7]:
temp['totalwords'] = temp['text'].str.count(' ') + 1

In [8]:
temp.columns


Index(['text', 'retweets_count', 'favorites_count', 'followers_count',
       'statuses_count', 'friends_count', 'verified', 'TweetID', 'totalwords'],
      dtype='object')

In [9]:
temp.drop(temp.columns[[0, 7]], axis=1, inplace=True)
df = temp

In [10]:
df

,retweets_count,favorites_count,followers_count,statuses_count,friends_count,verified,totalwords
0,3,0,3682,453535,3628,0,6
1,0,0,86,1016,284,0,1
2,3,1,1944,28234,1995,0,3
3,0,0,1,1072,0,0,14
4,0,0,13957,25311,10841,0,3
...,...,...,...,...,...,...,...
353963,0,0,34,1509,55,0,4
353964,0,0,89,11166,127,0,7
353965,3,0,1888,712,3086,0,4
353966,0,0,139,486,320,0,3


In [11]:
df.columns

Index(['retweets_count', 'favorites_count', 'followers_count',
       'statuses_count', 'friends_count', 'verified', 'totalwords'],
      dtype='object')

In [12]:
normalized_df=(df-df.min())/(df.max()-df.min())

In [13]:
normalized_df["TweetID"] = train_data["TweetID"]

In [14]:
normalized_df

,retweets_count,favorites_count,followers_count,statuses_count,friends_count,verified,totalwords,TweetID
0,0.000047,0.000000,2.553912e-04,0.055420,0.015291,0.0,0.128205,832509
1,0.000000,0.000000,5.965140e-06,0.000124,0.001197,0.0,0.000000,1388011
2,0.000047,0.000008,1.348399e-04,0.003450,0.008408,0.0,0.051282,63896
3,0.000000,0.000000,6.936210e-08,0.000131,0.000000,0.0,0.333333,979251
4,0.000000,0.000000,9.680868e-04,0.003093,0.045691,0.0,0.051282,1040049
...,...,...,...,...,...,...,...,...
353963,0.000000,0.000000,2.358311e-06,0.000184,0.000232,0.0,0.076923,142573
353964,0.000000,0.000000,6.173227e-06,0.001364,0.000535,0.0,0.153846,240866
353965,0.000047,0.000000,1.309556e-04,0.000087,0.013006,0.0,0.076923,1173763
353966,0.000000,0.000000,9.641331e-06,0.000059,0.001349,0.0,0.051282,929182
